In [3]:
import os
os.chdir("../")
%pwd


'c:\\Users\\IT STORE\\Desktop\\ML-Projects\\Text-Summarizer'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    overwrite_output_dir: bool
    num_train_epochs: int # Number of training epochs
    per_device_train_batch_size: int  # Batch size per training GPU/TPU
    save_steps: int   # Save model checkpoint every N steps
    save_total_limit: int  # Maximum number of checkpoints to save
    logging_steps: int # Log training information every N steps
    evaluation_strategy: str  # Evaluate model performance after each epoch
    metric_for_best_model: str  # Metric to monitor for early stopping (minimize loss)
    load_best_model_at_end: bool  # Load the best model based on the monitored metric
    warmup_steps: int  # Number of warmup steps for learning rate scheduler
    learning_rate: float  # Learning rate for the optimizer
    adam_epsilon: float  # Epsilon for Adam optimizer
    weight_decay: float  # Weight decay for optimizer



In [5]:
from src.constants import *
from src.utils import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        create_directories([config.root_dir])
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            overwrite_output_dir = params.overwrite_output_dir,
            num_train_epochs = params.num_train_epochs,
            per_device_train_batch_size= params.per_device_train_batch_size,
            save_steps = params.save_steps,
            save_total_limit = params.save_total_limit,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            metric_for_best_model = params.metric_for_best_model,
            load_best_model_at_end = params.load_best_model_at_end,
            warmup_steps = params.warmup_steps,
            learning_rate = params.learning_rate,
            adam_epsilon = params.adam_epsilon,
            weight_decay = params.weight_decay
        )

        
        return model_trainer_config

In [7]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import BartForConditionalGeneration, BertTokenizer
from datasets import load_dataset, load_from_disk
import torch

[2024-06-09 18:38:37,729: WARNING: module_wrapper: From c:\Users\IT STORE\anaconda3\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]
[2024-06-09 18:38:38,073: INFO: utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2024-06-09 18:38:38,075: INFO: utils: NumExpr defaulting to 8 threads.]


In [8]:
from transformers import TrainingArguments, Trainer
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import DataCollatorForSeq2Seq
from datasets import load_from_disk
import torch
import os

class ModelTrainer:
    def __init__(self, config):
        self.config = config
        
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"  # Corrected to torch.cuda.is_available()
        tokenizer = BartTokenizer.from_pretrained(self.config.model_ckpt)  # Assuming BartTokenizer is correct
        model_bert = BartForConditionalGeneration.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_bert)
        
        # loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            overwrite_output_dir=self.config.overwrite_output_dir,
            num_train_epochs=self.config.num_train_epochs,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            save_steps=self.config.save_steps,
            save_total_limit=self.config.save_total_limit,
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            metric_for_best_model=self.config.metric_for_best_model,
            load_best_model_at_end=self.config.load_best_model_at_end,
            warmup_steps=self.config.warmup_steps,
            learning_rate=self.config.learning_rate,
            adam_epsilon=self.config.adam_epsilon,
            weight_decay=self.config.weight_decay
        )


        
        model_trainer = Trainer(
            model=model_bert,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["train"],
            eval_dataset=dataset_samsum_pt["test"],
        )
        
        model_trainer.train()
        
        # Save model and tokenizer
        model_dir = os.path.join(self.config.root_dir, "bart-large-cnn-model")
        tokenizer_dir = os.path.join(self.config.root_dir, "bart-large-cnn-tokenizer")
        
        # Ensure the directories exist
        os.makedirs(model_dir, exist_ok=True)
        os.makedirs(tokenizer_dir, exist_ok=True)
        
        model_bert.save_pretrained(model_dir)
        tokenizer.save_pretrained(tokenizer_dir)


In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train()
except Exception as e:
    print(e)